In [2]:
import gradio as gr
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification 
import torch.nn.functional as F
import pandas as pd
import random

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the pretrained tokenizer and our trained model
model_checkpoint = "EleutherAI/gpt-neo-125m"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, add_prefix_space=True)
tokenizer.pad_token = tokenizer.eos_token

# Load the trained weights
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=5)
model_weights_path = "model_weights/imdb_sentiment_label_model.pth"
model.load_state_dict(torch.load(model_weights_path, map_location=DEVICE))
model.to(DEVICE)  # Move the model to the appropriate device
model.eval()

/home/ubuntu/ml-takehome/slingshot-venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of GPTNeoForTokenClassification were not initialized from the model checkpoint at EleutherAI/gpt-neo-125m and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_12021/3757544435.py:18: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for 

GPTNeoForTokenClassification(
  (transformer): GPTNeoModel(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(2048, 768)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPTNeoBlock(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPTNeoAttention(
          (attention): GPTNeoSelfAttention(
            (attn_dropout): Dropout(p=0.0, inplace=False)
            (resid_dropout): Dropout(p=0.0, inplace=False)
            (k_proj): Linear(in_features=768, out_features=768, bias=False)
            (v_proj): Linear(in_features=768, out_features=768, bias=False)
            (q_proj): Linear(in_features=768, out_features=768, bias=False)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPTNeoMLP(
          (c_fc): Linear(in_features=768, out_features=3072, bias=True)
          (c_proj): L

In [3]:
def predict_sentiment(text):
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="pt", truncation=True)
    inputs = inputs.to(DEVICE)

    # Set the model to evaluation mode
    model.eval()

    # Disable gradient calculations
    with torch.no_grad():
        # Get the model outputs
        outputs = model(**inputs)
        logits = outputs.logits

    # Convert logits to probabilities
    probabilities = F.softmax(logits, dim=-1)

    # Get the predicted classes for each token
    predicted_classes = torch.argmax(logits, dim=-1)

    # Map the predicted classes to sentiment labels
    sentiment_map = {0: "Negative", 1: "Mid-Negative", 2: "Neutral", 3: "Mid-Positive", 4: "Positive"}

    # Get the overall sentiment (mode of all token predictions)
    # 🚨
    overall_sentiment_class = torch.mode(predicted_classes[-1]).values.item()
    overall_sentiment = sentiment_map[overall_sentiment_class]

    # Calculate overall confidence (mean of highest probabilities for each token)
    overall_confidence = torch.mean(torch.max(probabilities[0], dim=1).values).item()

    # Decode tokens and pair with their predicted classes
    tokens = tokenizer.convert_ids_to_tokens(inputs.input_ids[0])
    # Found this 'Ġ' character is space token, so replacing it as such
    token_sentiments = [(i, token.replace('Ġ', ' '), predicted_classes[0, i].item()) 
                        for i, token in enumerate(tokens) 
                        if token not in [tokenizer.pad_token, '<|endoftext|>']]

    # Generate highlighted text
    highlighted_text = ""
    for i, token, sentiment_class in token_sentiments:
        sentiment = sentiment_map[sentiment_class]
        confidence_score = probabilities[0, i, sentiment_class].item()
        
        if sentiment in ["Very Positive", "Positive"]:
            color = f"rgba(0, 0, 255, {confidence_score})"  # Blue with varying opacity
        elif sentiment in ["Very Negative", "Negative"]:
            color = f"rgba(255, 165, 0, {confidence_score})"  # Orange with varying opacity
        else:
            color = "transparent"
        
        highlighted_text += f'<span style="background-color:{color}">{token}</span>'

    return overall_sentiment, highlighted_text

# Example usage
sample_text = "This is a great movie! I loved it."
sentiment, highlighted_text = predict_sentiment(sample_text)
print(f"Input text: {sample_text}")
print(f"Overall sentiment: {sentiment}")
print(f"Highlighted text: {highlighted_text}")

Input text: This is a great movie! I loved it.
Overall sentiment: Positive
Highlighted text: <span style="background-color:rgba(255, 165, 0, 0.53486567735672)"> This</span><span style="background-color:rgba(255, 165, 0, 0.5930820107460022)"> is</span><span style="background-color:rgba(255, 165, 0, 0.5326722860336304)"> a</span><span style="background-color:rgba(0, 0, 255, 0.9187238812446594)"> great</span><span style="background-color:rgba(0, 0, 255, 0.986683189868927)"> movie</span><span style="background-color:rgba(0, 0, 255, 0.9931069016456604)">!</span><span style="background-color:rgba(0, 0, 255, 0.9904682040214539)"> I</span><span style="background-color:rgba(0, 0, 255, 0.9896005988121033)"> loved</span><span style="background-color:rgba(0, 0, 255, 0.9960152506828308)"> it</span><span style="background-color:rgba(0, 0, 255, 0.9946004152297974)">.</span>


In [6]:
# Load and sample the dataset
def load_and_sample_data(file_path, n_samples=100):
    df = pd.read_csv(file_path)
    return df.sample(n=n_samples)

# Process the sampled data
def process_samples(samples):
    results = []
    for _, row in samples.iterrows():
        sentiment, highlighted_text = predict_sentiment(row['review'])
        results.append({
            'Review': highlighted_text,
            'Predicted Sentiment': sentiment,
            'True Sentiment': row['sentiment']
        })
    return pd.DataFrame(results)

# Create the Gradio interface
def create_interface():
    # Load and process the samples
    samples = load_and_sample_data('data/IMDB Dataset.csv')
    results = process_samples(samples)

    # Create the Gradio interface
    iface = gr.Interface(
        fn=predict_sentiment,
        inputs="text",
        outputs=["text", "html"],
        title="IMDB Sentiment Prediction",
        description="Enter text to predict sentiment and see highlights based on confidence.",
    )

    # Create HTML table for results
    table_html = "<table style='width:100%; border-collapse: collapse;'>"
    table_html += "<tr><th>Review</th><th>Predicted Sentiment</th><th>True Sentiment</th></tr>"
    
    for _, row in results.iterrows():
        predicted_sentiment = row['Predicted Sentiment']
        if predicted_sentiment in ["Positive", "Mid-Positive"]:
            sentiment_color = "rgba(0, 0, 255, 1)"  # Blue
        elif predicted_sentiment in ["Negative", "Mid-Negative"]:
            sentiment_color = "rgba(255, 165, 0, 1)"  # Orange
        else:
            sentiment_color = "transparent"
        
        table_html += f"<tr><td style='border: 1px solid #ddd; padding: 8px; word-wrap: break-word;'>{row['Review']}</td>"
        table_html += f"<td style='border: 1px solid #ddd; padding: 8px; background-color: {sentiment_color}; color: white;'>{predicted_sentiment}</td>"
        table_html += f"<td style='border: 1px solid #ddd; padding: 8px;'>{row['True Sentiment']}</td></tr>"
    
    table_html += "</table>"

    # Add the results table
    with gr.Blocks() as demo:
        gr.Markdown("# IMDB Sentiment Prediction")
        # Add text field for classification metrics
        gr.Markdown(f"""
        ## How to Read the Colors
        - Blue: Positive sentiment (darker blue indicates higher confidence)
        - Orange: Negative sentiment (darker orange indicates higher confidence)
        - No color: Neutral sentiment
        You'll see the model change its opinions as it reads the text
        """)
        iface.render()
        gr.Markdown("## Sample Results")
        gr.HTML(table_html)

    return demo

# Launch the interface
demo = create_interface()
demo.launch(share=True)

* Running on local URL:  http://127.0.0.1:7861
* Running on public URL: https://3a16c13a98545cb3cd.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
